<a href="https://colab.research.google.com/github/Tiru-Kaggundi/Adv_ML_project/blob/main/LM_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this milestone, we will be categorizing whether a given text belongs to one of the Indian languages from a set of {Hindi, Malayalam, Kannada, Tamil, Bengali}. For this we will create a Bag of Vectors for each language. Here are the steps that we aim to implement-

Collate the dataset for each language. We will use the Hindi, Kannada, Tamil, Malayalam, and Bengali Wikipedia articles to train our model. Use publicly available tokenizers (or generate a new one if possible - stretch goal) to create a Bag of Words Create a Bag of Words classifier using linear layers Train the Classifier

For Language Identification, a lot of work has been done to classify and identify languages with latin as the predominant script. Some work has also been done to identify Chinese, Japanese, and Korean languages. Jauhiaihen et al has done a detailed survey on the literature that exists for Language Identification tasks. Kerwin (2006) used character frequencies as feature vectors. In a feature vector, each feature vector f has its own integer value. Raw frequency and relative frequency for each feature is calculated for each language. For our project, we will be using something very similar.

There are several intuitive techniques that are used to classify languages- Position of words- Kumar et al. (2015) used the position of the current word in word-level LI. Dictionary of unique words: Unique word dictionaries include only those words of the language, that do not belong to the other languages targeted by the language identifier. Discriminating words Kolkus (2009) used the most relevant words for each language

Milestone 1: Develop text classifier of our own

Milestone 2: See text classificaiton using INLTK 

Milestone 3: Use transfer learning on INLTK for classification accuracy improvement